In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
from jax import config
config.update("jax_enable_x64", True)
config.update("jax_platform_name", "cpu")

import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]=".8"

In [3]:
from __future__ import annotations
import jax
import jax.numpy as jnp
import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from jax import vmap

from typing import Optional, List, Dict, Any, Union, Set, Tuple
from matplotlib.axes import Axes
from dataclasses import dataclass, field

from jaxley.io.graph import _build_module_scaffold

In [4]:
@jax.tree_util.register_dataclass
@dataclass
class MorphTree:
    """MorphTree is a custom cataclass that holds the node and edge attributes of a morphology.
    
    MorphTree is used to store the node and edge attributes of a morphology / jaxley Module
    as a pytree, to allow for easy manipulation of the Module / morphology parameters 
    using jax transformations.

    MorphTree also allows for easy conversion to and from pandas DataFrames, and networkx 
    DiGraphs, as well as basic convenience functions for plotting and renaming or reordering
    of nodes and edges.
    """
    node_attrs: Dict[int, Dict[str, Any]]
    edge_attrs: Dict[Tuple[int, int], Dict[str, Any]]
    global_attrs: Dict[str, Any] = field(default_factory=dict)

    @property
    def nodes(self) -> jnp.ndarray:
        """Returns the node indices as a jax array."""
        return jnp.array(list(self.node_attrs.keys())).astype(int)

    @property
    def edges(self) -> jnp.ndarray:
        """Returns the edge indices as a jax array."""
        return jnp.array(list(self.edge_attrs.keys())).astype(int)

    def __repr__(self) -> str:
        n_nodes = len(self.node_attrs)
        n_edges = len(self.edge_attrs)

        node_keys = list(next(iter(self.node_attrs.values())).keys())
        if len(self.edge_attrs) > 0:
            edge_keys = list(next(iter(self.edge_attrs.values())).keys())
        else:
            edge_keys = []

        node_attrs = node_keys if len(self.node_attrs) > 0 else []
        edge_attrs = edge_keys if len(self.edge_attrs) > 0 else []
        return f"MorphTree(nodes={n_nodes}*{node_attrs}, edges={n_edges}*{edge_attrs}, global={list(self.global_attrs.keys())})"
    
    def __iter__(self):
        """Allows unpacking of MorphTree as: node_attrs, edge_attrs, global_attrs = *tree"""
        yield self.node_attrs
        yield self.edge_attrs
        yield self.global_attrs
    
    def node(self, i: int) -> Dict[str, Any]:
        """Returns the node attributes for the node with index i."""
        return self.node_attrs[i]
    
    def edge(self, i: int, j: int) -> Dict[str, Any]:
        """Returns the edge attributes for the edge between nodes i and j."""
        return self.edge_attrs[i, j]
    
    def to_nx(self) -> nx.DiGraph:
        """Returns the MorphTree as a networkx DiGraph."""
        G = nx.DiGraph()
        G.add_nodes_from(self.node_attrs.items())
        G.add_edges_from((i, j, d) for (i, j), d in self.edge_attrs.items())
        G.graph.update(self.global_attrs)
        return G
    
    @staticmethod
    def from_nx(G: nx.DiGraph) -> MorphTree:
        """Returns a MorphTree from a networkx DiGraph."""
        node_attrs = {n: G.nodes[n] for n in G.nodes}
        edge_attrs = {(i, j): G.edges[i, j] for i, j in G.edges}
        return MorphTree(node_attrs, edge_attrs, G.graph)
    
    def to_pandas(self, return_global_attrs: bool = True) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Returns the MorphTree as a pandas DataFrame."""
        node_df = pd.DataFrame(self.node_attrs.values(), index=self.node_attrs.keys())
        edge_df = pd.DataFrame(self.edge_attrs.values(), index=self.edge_attrs.keys())
        edge_index = pd.MultiIndex.from_arrays(np.array(self.edges).T)
        edge_df = edge_df.set_index(edge_index)

        if return_global_attrs:
            return node_df, edge_df, pd.Series(self.global_attrs)
        return node_df, edge_df
    
    @staticmethod
    def from_pandas(node_df: pd.DataFrame, edge_df: pd.DataFrame, global_attrs: pd.Series = pd.Series()) -> MorphTree:
        """Returns a MorphTree from a pandas DataFrame."""
        node_attrs = node_df.to_dict(orient="index")
        edge_attrs = edge_df.to_dict(orient="index")
        return MorphTree(node_attrs, edge_attrs, global_attrs.to_dict())
    
    def plot(self, dims=(0,1), ax: Optional[Axes] = None, **kwargs: Any) -> Axes:
        """Uses networkx to plot the MorphTree.
        
        Args:
            dims: Dimensions to plot (0:x, 1:y, 2:z).
            ax: plt.Axes.
            **kwargs: kwargs for networkx.draw.

        Returns:
            The Axes object on which the MorphTree was plotted.
        """
        G = self.to_nx()
        pos = {}
        dims2axes = {0: "x", 1: "y", 2: "z"}
        for n, attr in G.nodes(data=True):
            if "x" in attr:  # assume y is also present
                pos[n] = (attr[dims2axes[dims[0]]], attr[dims2axes[dims[1]]])
        
        ax = ax if ax is not None else plt.gca()
        nx.draw(G, pos, with_labels=True, ax=ax, **kwargs)
        return ax

    def reindex_nodes(self, mapping: dict) -> MorphTree:
        """Reindexes the nodes of the MorphTree according to the mapping dictionary.
        
        Args:
            mapping: A dict mapping the old to new node indices.

        Returns:
            A new MorphTree with the nodes reindexed according to the mapping.
        """
        new_node_attrs = {mapping[i]: attrs for i, attrs in self.node_attrs.items()}
        
        new_edge_attrs = {}
        for (i, j), attrs in self.edge_attrs.items():
            new_edge_attrs[(mapping[i], mapping[j])] = attrs
            
        return MorphTree(new_node_attrs, new_edge_attrs, self.global_attrs)
    
    def reorder_tree(self, new_order: jnp.ndarray) -> MorphTree:
        """Reorders the nodes of the MorphTree according to the new order.
        
        Edges are flipped to ensure they are always in ascending order.
        
        Args:
            new_order: New node order. new_order[i] is the new index of node i.

        Returns:
            A new MorphTree with the nodes reordered according to the new order.
        """
        # TODO: check this does what I think it does, i.e. change the edge orientation
        # in order of appearance of the nodes in self.nodes.
        edges = np.array(self.edges)
        np_order = np.array(new_order)
        idx_i = np.where(edges[:,0] == np_order[:, None])[0]
        idx_j = np.where(edges[:,1] == np_order[:, None])[0]
        is_descending = ~(idx_i < idx_j)
        for (i,j) in edges[is_descending]:
            print(i,j)
            self.edge_attrs[j, i] = self.edge_attrs.pop((i, j))
        return self
    
    def subgraph(self, nodes: List[int]) -> MorphTree:
        """Returns a subset of nodes in the MorphTree.

        Edges are only included if both nodes are in the subgraph.
        
        Args:
            nodes: List of node indices to include in the subgraph.

        Returns:
            A new MorphTree containing only the specified nodes and their edges.
        """
        node_attrs_subset = {i: self.node_attrs[i] for i in nodes}
        edge_attrs_subset = {(i,j): attrs for (i,j), attrs in self.edge_attrs.items() if i in nodes and j in nodes}
        return MorphTree(node_attrs_subset, edge_attrs_subset, self.global_attrs)
    
# -----------------
# utility functions
# -----------------

def compute_xyz(G: nx.DiGraph,  length: float = 1.0,  spread: float = np.pi/8, spread_decay: float = 0.9,  twist: float = 0.0, xy_only: bool = True) -> dict[int, tuple[float, float, float]]:
    """Compute xyz coordinates for a tree-like appearance of a networkX graph in 2D or 3D.
    
    Handles branches implicitly since nodes in a branch have 1 child.

    Args:
        G: The Graph to compute node xyz coordinates for.
        length: The length of each edge.
        spread: The opening angle at which the edges spread out.
        spread_decay: Multiplicative decay factor for the opening angle / spread.
        twist: Add additional twisting. Means fewer overlapping nodes in 3D projections.
        xy_only: Whether to only compute the xy coordinates and fix the z-coordinate.

    Returns:
        A dictionary mapping node indices to xyz coordinates.
    """
    # TODO: Replace compute_xyz with this or vice versa, redundant!
    root = next(n for n, d in G.in_degree() if d == 0)
    pos = {root: (0.0, 0.0, 0.0)}

    def recurse(node, depth=1, theta=0.0, phi=np.pi/2):
        children = [n for n in G.successors(node) if n not in pos]
        if not children: return
        n = len(children)
        curr_spread = spread * (spread_decay ** (depth - 1))
        x0, y0, z0 = pos[node]
        phi = np.pi/2 if xy_only else phi
        base_theta = theta + depth * twist
        if n == 1:
            thetas, phis = [base_theta], [phi]
        else:
            if xy_only:
                thetas = np.linspace(base_theta - curr_spread/2, base_theta + curr_spread/2, n)
                phis = [phi] * n
            else:
                thetas = np.linspace(base_theta, base_theta + 2 * np.pi, n, endpoint=False)
                phis = [phi - curr_spread] * n
        for th, ph, child in zip(thetas, phis, children):
            x = x0 + length * np.sin(ph) * np.cos(th)
            y = y0 + length * np.sin(ph) * np.sin(th)
            z = z0 + length * np.cos(ph) * (not xy_only)
            pos[child] = (x, y, z)
            recurse(child, depth + 1, th, ph)

    recurse(root, theta=0.0, phi=np.pi/2)
    return pos

In [29]:
def pandas_to_nx(
    node_attrs: pd.DataFrame, edge_attrs: pd.DataFrame, global_attrs: pd.Series
) -> nx.DiGraph:
    """Convert node_attrs, edge_attrs and global_attrs from pandas datatypes to a NetworkX DiGraph.

    Args:
        node_attrs: DataFrame containing node attributes
        edge_attrs: DataFrame containing edge attributes
        global_attrs: Series containing global graph attributes

    Returns:
        A directed graph with nodes, edges and global attributes from the input data.
    """
    G = nx.from_pandas_edgelist(
        edge_attrs.reset_index(),
        source="level_0",
        target="level_1",
        edge_attr=True,
        create_using=nx.DiGraph(),
    )

    nx.set_node_attributes(G, node_attrs.to_dict(orient="index"))
    G.graph.update(global_attrs.to_dict())
    return G


def nx_to_pandas(G: nx.DiGraph) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series]:
    """Convert a NetworkX DiGraph to pandas datatypes.

    Args:
        G: Input directed graph

    Returns:
        Tuple containing:
        - DataFrame of node attributes
        - DataFrame of edge attributes
        - Series of global graph attributes
    """
    edge_df = nx.to_pandas_edgelist(G).set_index(["source", "target"])
    edge_df.index.names = [None, None]
    node_df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient="index")

    return node_df, edge_df, pd.Series(G.graph)


def swc_to_nx(fname: str, num_lines: Optional[int] = None) -> nx.DiGraph:
    """Read a SWC morphology file into a NetworkX DiGraph.

    Args:
        fname: Path to the SWC file
        num_lines: Number of lines to read from the file

    Returns:
        A directed graph representing the morphology where:
        - Nodes have attributes: id, x, y, z, r (radius)
        - Edges represent parent-child relationships
    """
    i_id_xyzr_p = np.loadtxt(fname)[:num_lines]

    graph = nx.DiGraph()
    for i, id, x, y, z, r, p in i_id_xyzr_p.tolist():  # tolist: np.float64 -> float
        graph.add_node(int(i), **{"id": int(id), "x": x, "y": y, "z": z, "r": r})
        if p != -1:
            graph.add_edge(int(p), int(i))
    return graph


def _split_branches(
    branches: list[list[int]], split_edges: list[tuple[int, int]]
) -> list[list[int]]:
    """Split branches at the given edges.

    Args:
        branches: List of branches, each represented as list of nodes.
        split_edges: List of edges between nodes where tracing is discontinous.

    Returns:
        An updated list of branches.
    """
    for p, n in split_edges:
        for i, branch in enumerate(branches):
            if n in branch:
                split_idx = branch.index(n)
                branches[i : i + 1] = [branch[:split_idx], branch[split_idx:]]
                break
    return branches


def _find_swc_tracing_interruptions(G: nx.DiGraph) -> list[tuple[int, int]]:
    """Identify discontinuities in the swc tracing order.

    Some swc files contain artefacts, where tracing of the same neurite was done
    in disconnected pieces. NEURON swc reader introduce a break in the trace at these
    points, since they parse the file in order. This leads to split branches, which
    should be one. This function identifies these points in the graph.

    Example swc file:
    # branch 1
    1 1 0.0 0.0 0.0 1.0 -1
    2 1 1.0 0.0 0.0 1.0 1
    3 1 2.0 0.0 0.0 1.0 2
    # branch 2
    4 2 3.0 1.0 0.0 1.0 3
    5 2 4.0 2.0 0.0 1.0 4
    # branch 3
    6 3 3.0 -1.0 0.0 1.0 3
    7 3 4.0 -2.0 0.0 1.0 6
    8 3 5.0 -3.0 0.0 1.0 7
    # ammend branch 2
    9 4 5.0 3.0 0.0 1.0 5

    Args:
        G: NetworkX graph tracing of swc file.

    Returns:
        An array of edges where tracing is discontinous.
    """
    degree_is_2 = lambda n: G.out_degree(n) + G.in_degree(n) == 2

    interrupt_edges = []
    for n in G.nodes:
        if len(parents := list(G.predecessors(n))) > 0:
            p = parents[0]
            if p != n - 1 and degree_is_2(n) and degree_is_2(p):
                interrupt_edges.append((p, n))
    return interrupt_edges


def list_branches(
    G,
    return_branchpoints: bool = False,
    ignore_swc_tracing_interruptions=True,
    relevant_ids: List[int] = [1, 2, 3, 4],
    max_len: Optional[float] = None,
) -> Union[List[List[int]], Tuple[List[List[int]], Set[int], List[Tuple[int, int]]]]:
    """Get all uninterrupted paths in the traced morphology (i.e. branches).

    The graph is traversed depth-first starting from the first found leaf node.
    Nodes are considered to be part of a branch if they have only one parent and one
    child, which are both of the same type (i.e. have the same `id`). Nodes which are
    branchpoints or leafs are considered start / end points of a branch. A branchpoint
    can start multiple branches.

    Args:
        G: NetworkX graph tracing of swc file.
        return_branchpoints: Whether to return the branchpoints and edges between them
            seperately.
        ignore_swc_tracing_interruptions: Whether to ignore discontinuities in the swc
            tracing order. If False, this will result in split branches at these points.
        relevant_ids: All type ids that are not in this list will be ignored for
            tracing the morphology. This means that branches which have multiple type
            ids (which are not in `relevant_ids`) will be considered as one branch.
            Defaults to `[1, 2, 3, 4]`.

    Returns:
        A list of linear paths in the graph. Each path is represented as list of nodes.
    """
    G = G.to_undirected()
    branches = []
    branchpoints = set()
    visited = set()

    was_visited = lambda n1, n2: (n1, n2) in visited or (n2, n1) in visited
    id_of = lambda n: G.nodes[n]["id"]
    is_soma = lambda n: id_of(n) == 1
    soma_nodes = lambda: [i for i, n in G.nodes.items() if n["id"] == 1]

    def is_branchpoint_or_tip(n: int) -> bool:
        if G.degree(n) == 2:
            i, j = G.neighbors(n)
            # trace dir matters here! For segment with node IDs: [1, 1, 2, 2]
            # -> [[1,1], [1,2,2]]
            # <- [[2,2], [2,1,1]]
            return not same_id(n, j)

        is_leaf = G.degree(n) <= 1
        is_branching = G.degree(n) > 2
        return is_leaf or is_branching

    def same_id(n1: int, n2: int) -> bool:
        has_id = lambda n: id_of(n) in relevant_ids if "id" in G.nodes[n] else False
        if has_id(n1) and has_id(n2):
            return id_of(n1) == id_of(n2)
        return True

    def walk_path(start: int, succ: int) -> List[int]:
        """Walk from start to succ, recording new nodes until a branching node is hit."""
        path = [start, succ]
        visited.add((start, succ))

        while G.degree(succ) == 2:
            next_node = next(n for n in G.neighbors(succ) if n != path[-2])

            if was_visited(succ, next_node) or is_branchpoint_or_tip(succ):
                break

            path.append(next_node)
            visited.add((succ, next_node))
            succ = next_node

        return path

    leaf = next(n for n in G.nodes if G.degree(n) == 1)
    single_soma = len(soma_nodes()) == 1
    for node in nx.dfs_tree(G, leaf):
        if single_soma and is_soma(node):
            branches.append([node])

        elif is_branchpoint_or_tip(node):
            branchpoints.add(node)
            for succ in G.neighbors(node):
                if not was_visited(node, succ):
                    branches.append(walk_path(node, succ))

    # split branches
    if not ignore_swc_tracing_interruptions:  # TODO: fix!
        split_edges = _find_swc_tracing_interruptions(G)
        branches = _split_branches(branches, split_edges)
        branchpoints.update(set(p for (p, n) in split_edges))

    # TODO: add max_len
    if max_len is not None:
        raise NotImplementedError("max_len not implemented")

    if return_branchpoints:
        branchpoint_edges = sum([list(G.edges(n)) for n in branchpoints], [])
        return branches, branchpoints, branchpoint_edges
    return branches


def _add_missing_swc_attrs(G) -> nx.DiGraph:
    """Add missing swc attributes to a SWC graph.

    Allows to specify morphology from just edges.

    Args:
        G: The SWC graph to add missing attributes to.

    Returns:
        The SWC graph with missing attributes set to their defaults.
    """
    defaults = {"id": 0, "r": 1}

    available_keys = G.nodes[next(iter(G.nodes()))].keys()
    xyz = compute_xyz(G) if "x" not in available_keys else {}
    for n, (x, y, z) in xyz.items():
        G.nodes[n]["x"] = x
        G.nodes[n]["y"] = y
        G.nodes[n]["z"] = z

    for key in set(defaults.keys()).difference(available_keys):
        nx.set_node_attributes(G, defaults[key], key)
    return G


def compartmentalize(
    swc_graph: nx.DiGraph,
    ncomp: int = 1,
    min_radius: Optional[float] = None,
    max_len: Optional[float] = None,
    ignore_swc_tracing_interruptions: bool = True,
    relevant_type_ids: List[int] = [1, 2, 3, 4],
) -> nx.DiGraph:
    """Return a networkX graph that indicates the compartment structure.

    Build a new graph made up of compartments in every branch. These compartments are
    spaced at equidistant points along the branch. Node attributes, like radius are
    linearly interpolated along its length.

    Example: 4 compartments | edges = - | nodes = o | comp_nodes = x
    o-----------o----------o---o---o---o--------o
    o-------x---o----x-----o--xo---o---ox-------o

    This function returns a nx:DiGraph. The graph is directed only because every
    compartment tracks the xyzr coordinates of the associated SWC file. These xyzr
    coordinates are ordered by the order of the traversal of the swc_graph. In later
    methods (e.g. build_solve_graph), we traverse the `comp_graph` and mostly ignore
    the directionality of the edges, but we only use the directionality to reverse the
    xyzr coordinates if necessary.

    Args:
        swc_graph: Graph generated by `to_swc_graph()`.
        ncomp: How many compartments per branch to insert.
        root: The root branch from which to start tracing the nodes. This defines the
            branch indices.
        min_radius: Minimal radius for each compartment.
        max_len: Maximal length for each branch. Longer branches are split into
            separate branches.
        ignore_swc_tracing_interruptions: If `False`, it this function automatically
            starts a new branch when a section is traced with interruptions.
        relevant_type_ids: All type ids that are not in this list will be ignored for
            tracing the morphology. This means that branches which have multiple type
            ids (which are not in `relevant_type_ids`) will be considered as one branch.
            If `None`, we default to `[1, 2, 3, 4]`.

    Returns:
        Graph of the compartmentalized morphology.

    Example usage
    ^^^^^^^^^^^^^

    ::

        from jaxley.io.graph swc_to_nx
        swc_graph = swc_to_nx("path_to_swc.swc")
        comp_graph = compartmentalize(swc_graph, ncomp=1)
    """
    G = _add_missing_swc_attrs(swc_graph)

    branches = list_branches(
        G,
        ignore_swc_tracing_interruptions=ignore_swc_tracing_interruptions,
        max_len=max_len,
        relevant_ids=relevant_type_ids,
    )
    nodes_df = nx_to_pandas(G)[0].astype(float)

    # create new set of indices which arent already used as node indices to label comps
    existing_inds = set(nodes_df.index)
    num_new_inds = len(branches) * ncomp
    proposed_inds = set(range(num_new_inds + len(existing_inds)))
    proposed_comp_inds = list(
        proposed_inds - existing_inds
    )  # avoid overlap w. node indices

    # identify tip nodes (degree == 1 -> node appears only once in edges)
    nodes_in_edges, node_counts_in_edges = np.unique(G.edges, return_counts=True)
    tip_node_inds = nodes_in_edges[node_counts_in_edges == 1]

    # collect comps and comp_edges
    branch_nodes, branch_edges = [], []
    xyzr = []
    for i, branch in enumerate(branches):
        node_attrs = nodes_df.loc[branch]
        xyzr_i = node_attrs[["x", "y", "z", "r"]]
        xyzr.append(xyzr_i.values)

        compute_edge_lens = lambda x: (x.diff(axis=0).fillna(0) ** 2).sum(axis=1) ** 0.5
        edge_lens = compute_edge_lens(xyzr_i[["x", "y", "z"]])
        node_attrs["l"] = edge_lens.cumsum()  # path length

        # For single-point somatata, we set l = 2*r this ensures
        # A_cylinder = 2*pi*r*l = 4*pi*r^2 = A_sphere.
        if len(branch) == 1:
            node_attrs = node_attrs.loc[branch * 2]  # duplicate soma node
            radius = node_attrs["r"].iloc[0]
            node_attrs["l"] = np.array([0, 2 * radius])

        branch_id = node_attrs["id"].iloc[
            -1
        ]  # TODO: how to handle mult. ids in branch!
        branch_len = max(node_attrs["l"])
        comp_len = branch_len / ncomp
        comp_locs = list(np.linspace(comp_len / 2, branch_len - comp_len / 2, ncomp))

        # Create node indices and attributes
        # branch_inds, branchpoint, comp_id, comp_len
        branch_tips = branch[0], branch[-1]
        branch_tip_attrs = [
            [i, True, node_attrs["id"].iloc[0], 0],
            [i, True, node_attrs["id"].iloc[-1], 0],
        ]
        comp_attrs = [[i, False, branch_id, comp_len]] * ncomp

        comp_inds = proposed_comp_inds[i * ncomp : (i + 1) * ncomp]
        comp_inds = np.array([branch_tips[0], *comp_inds, branch_tips[1]])
        comp_attrs = [branch_tip_attrs[0]] + comp_attrs + [branch_tip_attrs[1]]
        comp_attrs = np.hstack([comp_inds[:, None], comp_attrs])

        x = np.array([0] + comp_locs + [branch_len])
        xp = np.array(node_attrs["l"].values)
        fp = np.array(xyzr_i.values)

        # TODO: interpolate **r** differently!
        interpolated_coords = np.column_stack(
            [np.interp(x, xp, fp[:, i]) for i in range(fp.shape[1])]
        )

        # Combine interpolated coordinates with existing attributes
        comp_attrs = np.hstack([comp_attrs, interpolated_coords])

        # remove tip nodes
        comp_attrs = comp_attrs[1:] if branch_tips[0] in tip_node_inds else comp_attrs
        comp_attrs = comp_attrs[:-1] if branch_tips[1] in tip_node_inds else comp_attrs

        # Store edges and nodes
        branch_edges.append(list(zip(comp_attrs[:-1, 0], comp_attrs[1:, 0])))
        branch_nodes.append(comp_attrs)

    branch_nodes = np.concatenate(branch_nodes)
    comp_attrs_keys = ["idx", "branch", "branchpoint", "id", "l", "x", "y", "z", "r"]
    comp_df = pd.DataFrame(branch_nodes, columns=comp_attrs_keys)

    int_cols = ["idx", "branch", "id"]
    comp_df[int_cols] = comp_df[int_cols].astype(int)

    bool_cols = ["branchpoint"]
    comp_df[bool_cols] = comp_df[bool_cols].astype(bool)

    # threshold radius
    if min_radius is not None:
        comp_df["r"] = np.maximum(comp_df["r"], min_radius)

    # drop duplicated branch nodes
    comp_df = comp_df.drop_duplicates(subset=["idx"])
    comp_df = comp_df.set_index("idx")

    # create comp edges
    comp_edges = sum(branch_edges, [])
    comp_edges_df = pd.DataFrame(index=pd.MultiIndex.from_tuples(comp_edges))
    comp_edges_df["synapse"] = False
    comp_edges_df["comp_edge"] = True

    global_attrs = pd.Series({"xyzr": xyzr})
    G = pandas_to_nx(comp_df, comp_edges_df, global_attrs)
    G = nx.relabel_nodes(G, {n: i for i, n in enumerate(G.nodes)})
    return G


def _add_jaxley_meta_data(G: nx.DiGraph) -> nx.DiGraph:
    """Add attributes to and rename existing attributes of the compartalized morphology.

    Makes the imported and compartmentalized morphology compatible with jaxley.
    """
    nodes_df, edge_df, global_attrs = nx_to_pandas(G)
    module_global_attrs = pd.Series({"channels": {}, "synapses": {}, "group_names": []})
    global_attrs = pd.concat([global_attrs, module_global_attrs])

    # rename/reformat existing columns
    group_ids = {0: "undefined", 1: "soma", 2: "axon", 3: "basal", 4: "apical"}
    for group_id, group_name in group_ids.items():
        where_group = nodes_df["id"] == group_id
        if where_group.any():
            global_attrs["group_names"].append(group_name)
            nodes_df[group_name] = where_group
    nodes_df = nodes_df.drop(columns=["id"])
    module_col_names = {"r": "radius", "l": "length", "branch": "branch_index"}
    nodes_df = nodes_df.rename(module_col_names, axis=1)

    # new columns
    nodes_df["capacitance"] = 1.0
    nodes_df["v"] = -70.0
    nodes_df["axial_resistivity"] = 1000.0
    # TODO: rename to cell_index > cell, comp_index > comp, branch_index > branch
    nodes_df["comp_index"] = np.arange(len(nodes_df))
    nodes_df["cell_index"] = 0

    return pandas_to_nx(nodes_df, edge_df, global_attrs)


def _replace_branchpoints_with_edges(G: nx.DiGraph) -> nx.DiGraph:
    """Replace branchpoint nodes with edges connecting parent and children."""
    G.add_edges_from([(i, j, {"branch_edge": False}) for i, j in G.edges])
    branch_edge_attrs = {"comp_edge": True, "synapse": False, "branch_edge": True}

    branchpoints = [n for n in G.nodes if G.nodes[n]["branchpoint"]]
    for n in branchpoints:
        parent = next(G.predecessors(n))
        children = list(G.successors(n))

        # remove branchpoint and connect parent to children
        G.remove_node(n)
        G.add_edges_from([(parent, c) for c in children], **branch_edge_attrs)

    for i, n in enumerate(G.nodes):
        G.nodes[n].pop("branchpoint")
        # TODO: Can we skip relabeling comps and reindexing the dataframe?
        G.nodes[n]["comp_index"] = i
    G = nx.relabel_nodes(G, {n: i for i, n in enumerate(G.nodes)})

    return G


def _compute_branch_parents(
    node_df: pd.DataFrame, edge_df: pd.DataFrame
) -> List[List[int]]:
    """Compute the parent structure of the branch graph (for each cell).

    Args:
        node_df: The node dataframe of the graph.
        edge_df: The edge dataframe of the graph.

    Returns:
        The parent structure of the branch graph for each cell.
    """
    branch_edge_inds = edge_df.index[edge_df["branch_edge"]]
    parent_inds = branch_edge_inds.get_level_values(0)
    child_inds = branch_edge_inds.get_level_values(1)

    branch_edges = pd.DataFrame(
        {
            "parent_branch": node_df["branch_index"].loc[parent_inds].values,
            "child_branch": node_df["branch_index"].loc[child_inds].values,
        }
    )

    acc_parents = []
    parent_branch_inds = branch_edges.set_index("child_branch").sort_index()[
        "parent_branch"
    ]

    for branch_inds in node_df.groupby("cell_index")["branch_index"].unique():
        root_branch_idx = branch_inds[0]
        parents = parent_branch_inds.loc[branch_inds[1:]] - root_branch_idx
        acc_parents.append([-1] + parents.tolist())
    return acc_parents


def _build_module(G: nx.DiGraph) -> jx.Module:
    """Build a Module from a compartmentalized morphology.

    This function builds a Module from a nx.DiGraph that has been compartmentalized.

    Args:
        G: The graph to build the Module from.

    Returns:
        The Module built from the graph.
    """
    node_df, edge_df, global_attrs = nx_to_pandas(G)

    nodes_per_branch = node_df["branch_index"].value_counts()
    assert (
        nodes_per_branch.nunique() == 1
    ), "`from_graph()` does not support a varying number of compartments in each branch."

    acc_parents = _compute_branch_parents(node_df, edge_df)
    module = _build_module_scaffold(
        node_df, parent_branches=acc_parents, xyzr=global_attrs["xyzr"]
    )

    node_df.columns = [
        "global_" + col if "local" not in col and "index" in col else col
        for col in node_df.columns
    ]

    synapse_edges = edge_df[edge_df.synapse]

    # set column-wise. preserves cols not in df.
    module.nodes[node_df.columns] = node_df
    module.edges = synapse_edges if not synapse_edges.empty else module.edges

    # add all the extra attrs
    module.synapses = global_attrs["synapses"]
    module.channels = global_attrs["channels"]
    module.group_names = global_attrs["group_names"]
    module.membrane_current_names = [c.current_name for c in module.channels]
    module.synapse_names = [s._name for s in module.synapses]

    return module


def nx_to_module(swc_graph: nx.DiGraph, ncomp: int = 1) -> jx.Module:
    comp_graph = compartmentalize(swc_graph, ncomp=ncomp)
    comp_graph = _add_jaxley_meta_data(comp_graph)
    solve_graph = _replace_branchpoints_with_edges(comp_graph)
    module = _build_module(solve_graph)
    return module


def read_swc(fname: str) -> jx.Module:
    swc_graph = swc_to_nx(fname)
    return nx_to_module(swc_graph)


# def module_to_nx(module: jx.Module) -> nx.DiGraph:
#     global_attrs = {}
#     global_attrs["group_names"] = module.group_names
#     global_attrs["synapses"] = module.synapses
#     global_attrs["channels"] = module.channels

#     node_attrs = module.nodes
#     edge_attrs = module.edges

#     node_attrs = node_attrs.drop(columns=[col for col in node_attrs.columns if col.startswith("local_")])

#     return pandas_to_nx(node_attrs, edge_attrs, global_attrs)

In [30]:
cell = read_swc("../jaxley/tests/swc_files/morph_ca1_n120.swc")

In [114]:
# G = morph_tree.to_nx()
# branches = list_branches(morph_tree)

# min_len = 1e-5
# max_len = 100
# xyz = lambda i: np.array([G.nodes[i][k] for k in ["x", "y", "z"]])
# dl = lambda i,j: np.sqrt(((xyz(i) - xyz(j)) ** 2).sum())

# for branch in branches:
#     edge_lens = np.array([dl(i,j) for i,j in zip(branch[:-1], branch[1:])])
#     edge_lens = np.where(edge_lens < min_len, min_len, edge_lens)
#     cum_edge_lens = np.concatenate([[0], edge_lens.cumsum()])
#     local_branch_inds = cum_edge_lens // max_len
    
#     # Find indices where local_branch_inds changes
#     change_points = np.where(np.diff(local_branch_inds) != 0)[0]
#     # Get the edges (pairs of nodes) where the change occurs
#     change_edges = [(branch[i], branch[i+1]) for i in change_points]
#     print(change_edges)
    


In [168]:
morph_tree = swc_to_morphtree("../jaxley/tests/swc_files/morph_ca1_n120.swc")
morph_comps = compartmentalize(morph_tree, ncomp=1)
morph_comps = _add_jaxley_meta_data(morph_comps)
morph_comps = _replace_branchpoints_with_edges(morph_comps)
cell = _build_module(morph_comps)

In [171]:
for n, attrs in morph_comps.node_attrs.items():
    attrs["test"] = jnp.ones((1,1))

In [180]:
jax.tree_util.tree_map(lambda *args: jnp.array(args), *morph_comps.node_attrs.values())

{'apical': Array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True, False,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, Fal

In [117]:
class DummyChannel:
    def __init__(self, name = None):
        self.name = self.__class__.__name__ if name is None else name
        self.params = {f"gbar_{self.name}": 1.0, f"e_{self.name}": 0.0, f"nn_weights_{self.name}": jnp.ones((10, 10))}
        self.states = {f"m_{self.name}": 0.5, f"h_{self.name}": 0.5}

def tree_insert(tree, inds, channel):
    # TODO: Should Module and MorphTree be separate or the same thing?
    tree.global_attrs["channels"][channel.name] = channel
    for i in inds:
        tree.node_attrs[i].update(channel.params)
        tree.node_attrs[i].update(channel.states)

def tree_set(tree, key, value):
    nodes_df, edge_df = tree.to_pandas()
    # if ... if, since param could be in both nodes and edges
    if key in nodes_df.columns:
        valid = ~nodes_df[key].isna() & ~nodes_df.branchpoint
        nodes_df.loc[valid, key] = value
    if key in edge_df.columns:
        valid = ~edge_df[key].isna() & edge_df.synapse
        edge_df.loc[valid, key] = value
    return MorphTree.from_pandas(nodes_df, edge_df, pd.Series(tree.global_attrs))

    # # not vectorized, but allows to modify in place
    # nodes_df, edge_df = tree.to_pandas()
    # if key in nodes_df.columns:
    #     for i in tree.node_attrs:
    #         if key in tree.node_attrs[i]:
    #             tree.node_attrs[i][key] = value
    # if key in edge_df.columns:
    #     for i, j in tree.edge_attrs:
    #         if key in tree.edge_attrs[(i,j)]:
    #             tree.edge_attrs[(i,j)][key] = value


#TODO: split/merge similar to equinox
def merge_trees(trees: List[MorphTree]) -> MorphTree:
    tree = trees[0]
    for t in trees[1:]:
        tree.node_attrs.update(t.node_attrs)
        tree.edge_attrs.update(t.edge_attrs)
    return tree

# def split_tree(tree: MorphTree, nodes: List[int]) -> List[MorphTree]:
#     pass

def tree_to_pytree(tree):
    nodes_df, edge_df = tree.to_pandas()
    jax_nodes, jax_edges = {}, {}
    
    node_data = nodes_df.loc[~nodes_df.branchpoint]
    node_data = node_data.drop(columns=["branch", "branchpoint", "groups"])
    for key in node_data.columns:
        values = jnp.array(node_data.loc[~node_data[key].isna(), key].to_list())
        jax_nodes[key] = values

    edge_data = edge_df.loc[edge_df.synapse]
    edge_data = edge_data.drop(columns=["comp_edge", "synapse"])
    for key in edge_data.columns:
        values = jnp.array(edge_data.loc[~edge_data[key].isna(), key].to_list())
        jax_edges[key] = values
    return jax_nodes, jax_edges

def tree_view(tree: MorphTree, nodes: List[int]) -> MorphTree:
    return tree.subgraph(nodes)

class DummySynapse:
    def __init__(self, name = None):
        self.name = self.__class__.__name__ if name is None else name
        self.params = {f"gbar_{self.name}": 1.0, f"e_{self.name}": 0.0, f"nn_weights_{self.name}": jnp.ones((10, 10))}
        self.states = {f"m_{self.name}": 0.5, f"h_{self.name}": 0.5}

def tree_connect(tree, pre, post, synapse):
    # TODO: map / vectorize this
    tree.global_attrs["synapses"][synapse.name] = synapse
    for i, j in zip(pre, post):
        if (i,j) not in tree.edge_attrs:
            tree.edge_attrs[(i,j)] = {"comp_edge": False}
        tree.edge_attrs[i, j]["synapse"] = True
        tree.edge_attrs[i, j].update(synapse.params)
        tree.edge_attrs[i, j].update(synapse.states)


# There can only be one edge per pair of nodes. (or use MultiDiGraph).
# This means all synapses need to live in the same edge (i,j)
# -> treat synapses more like channels, i.e. multiple channels per row in nodes -> multiple synapses per edge.
# downside cannot connect i and j with the same synapse twice, but can do if one synapse is named differently.
# think about how to handle if i,j is a comp_edge and also connects via synapses

# TODO: node and edge attrs as list or dict?
# - pro: one can change node / edge idx without changing touching attrs, since pos of node_idx -> pos node_attr
# - con: hard to index into node / edge attrs

# TODO: get rid of xyzr in most usecases and use the comp / node xyzrs!? -> simplifies all plotting to networkx!
    

In [118]:
tree_insert(morph_comps, [0, 2], DummyChannel("test1"))
tree_insert(morph_comps, [0, 2], DummyChannel("test2"))

tree_connect(morph_comps, [0, 1], [3, 2], DummySynapse("test1"))
tree_connect(morph_comps, [0, 1], [1, 2], DummySynapse("test2"))

# tree_set(morph_comps, "gbar_test1", 123)
# tree_set(morph_comps, "gbar_test2", 123)

# subtree = tree_view(morph_comps, list(range(0,2)))
# subtree = tree_set(subtree, "length", 99.99) # <-- setting on views works
# morph_comps = merge_trees([morph_comps, subtree])

In [356]:
from jaxley.io.graph import build_compartment_graph, to_swc_graph, _trace_branches, _remove_branch_points

testcases = [ 
"morph_3_types_single_point_soma.swc",
"morph_3_types.swc",
"morph_interrupted_soma.swc",
"morph_soma_both_ends.swc",
"morph_somatic_branchpoint.swc",
"morph_non_somatic_branchpoint.swc", # no soma!
"morph_ca1_n120_single_point_soma.swc",
"morph_ca1_n120.swc",
"morph_l5pc_with_axon.swc",
"morph_allen_485574832.swc",
]

jx_graph = to_swc_graph("../jaxley/tests/swc_files/"+testcases[-3])
# jx_comps = build_compartment_graph(jx_graph.copy(), ncomp=1)

morph_tree = swc_to_morph_tree("../jaxley/tests/swc_files/"+testcases[-3])
morph_comps = compartmentalize(morph_tree, num_comps=1)
morph_comps = _add_meta_data(morph_comps)

In [244]:
morph_comps.to_pandas()[0]["nn_weights_test1"]

0      [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...
1                                                    NaN
2      [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...
3                                                    NaN
4                                                    NaN
                             ...                        
228                                                  NaN
229                                                  NaN
230                                                  NaN
231                                                  NaN
232                                                  NaN
Name: nn_weights_test1, Length: 233, dtype: object

In [260]:
jax_nodes, jax_edges = tree_to_pytree(morph_comps)

In [1582]:
from jaxley.io.graph import build_compartment_graph, to_swc_graph, _trace_branches

testcases = [ 
"morph_3_types_single_point_soma.swc",
"morph_3_types.swc",
"morph_interrupted_soma.swc",
"morph_soma_both_ends.swc",
"morph_somatic_branchpoint.swc",
"morph_non_somatic_branchpoint.swc", # no soma!
"morph_ca1_n120_single_point_soma.swc",
"morph_ca1_n120.swc",
"morph_l5pc_with_axon.swc",
"morph_allen_485574832.swc",
]

for i, testcase in enumerate(testcases):
    jx_graph = to_swc_graph("../jaxley/tests/swc_files/"+testcase)

    morph_branches = list_branches(MorphTree.from_nx(jx_graph.copy()))
    morph_branch_nodes = [np.sort(b) for b in morph_branches]

    # do jx_trace after morph_traces, since jax_trace modifies the graph
    jx_branches = _trace_branches(jx_graph.copy())[1]
    jx_branch_nodes = [np.sort(np.unique(b[:, :-1])) for b in jx_branches]
    if i in [0,6,9]: # single soma
        jx_branch_nodes = [b-1 for b in jx_branch_nodes]

    morph_eq_jx = []

    for i, b in enumerate(jx_branch_nodes):
        for j, mb in enumerate(morph_branch_nodes):
            if len(b) == len(mb):
                if np.allclose(b, mb):
                    morph_eq_jx.append((i,j))
                    break
    if len(morph_eq_jx) > 0:
        diff_morph_branches = [b for i, b in enumerate(morph_branch_nodes) if i not in np.array(morph_eq_jx)[:,1]]
        diff_jx_branches = [b for j, b in enumerate(jx_branch_nodes) if j not in np.array(morph_eq_jx)[:,0]]
    else:
        print("No branches are equal")
        diff_morph_branches = morph_branch_nodes
        diff_jx_branches = jx_branch_nodes
    
    # single soma handled differently and will lead to 1 diff branch
    print(f"testcase {testcase}: {len(diff_morph_branches)}, {len(diff_jx_branches)}")

testcase morph_3_types_single_point_soma.swc: 1, 1
testcase morph_3_types.swc: 0, 0
testcase morph_interrupted_soma.swc: 0, 0
testcase morph_soma_both_ends.swc: 0, 0
testcase morph_somatic_branchpoint.swc: 0, 0
testcase morph_non_somatic_branchpoint.swc: 0, 0
testcase morph_ca1_n120_single_point_soma.swc: 1, 1
testcase morph_ca1_n120.swc: 0, 0
testcase morph_l5pc_with_axon.swc: 0, 0
testcase morph_allen_485574832.swc: 1, 1


In [867]:
# from jaxley.io.graph import build_compartment_graph, to_swc_graph, _trace_branches

# testcases = [ 
# "morph_3_types_single_point_soma.swc",
# "morph_3_types.swc",
# "morph_interrupted_soma.swc",
# "morph_soma_both_ends.swc",
# "morph_somatic_branchpoint.swc",
# "morph_non_somatic_branchpoint.swc", # no soma!
# "morph_ca1_n120_single_point_soma.swc",
# "morph_ca1_n120.swc",
# "morph_l5pc_with_axon.swc",
# "morph_allen_485574832.swc",
# ]

# jx_graph = to_swc_graph("../jaxley/tests/swc_files/"+testcases[2])

# morph_branches = list_branches(MorphTree.from_nx(jx_graph.copy()))
# morph_branch_nodes = [np.sort(b) for b in morph_branches]

# # do jx_trace after morph_traces, since jax_trace modifies the graph
# jx_branches = _trace_branches(jx_graph.copy())[1]
# jx_branch_nodes = [np.sort(np.unique(b[:, :-1])) for b in jx_branches]
# # jx_branch_nodes = [b-1 for b in jx_branch_nodes]


# morph_eq_jx = []

# for i, b in enumerate(jx_branch_nodes):
#     for j, mb in enumerate(morph_branch_nodes):
#         if len(b) == len(mb):
#             if np.allclose(b, mb):
#                 morph_eq_jx.append((i,j))
#                 break
# if len(morph_eq_jx) > 0:
#     diff_morph_branches = [b for i, b in enumerate(morph_branch_nodes) if i not in np.array(morph_eq_jx)[:,1]]
#     diff_jx_branches = [b for j, b in enumerate(jx_branch_nodes) if j not in np.array(morph_eq_jx)[:,0]]
# else:
#     print("No branches are equal")
#     diff_morph_branches = morph_branch_nodes
#     diff_jx_branches = jx_branch_nodes

# jx_subgraph = jx_graph.subgraph(np.unique(np.hstack(diff_morph_branches)))

# # Get node positions and colors
# pos = {node: (jx_subgraph.nodes[node]['x'], jx_subgraph.nodes[node]['y']) for node in jx_subgraph.nodes()}

# # Create figure with 1x3 subplots
# fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# # Plot 1: Node indices
# node_colors = [jx_subgraph.nodes[node]['id'] for node in jx_subgraph.nodes()]
# nx.draw(jx_subgraph, pos=pos, node_color=node_colors, cmap='viridis', with_labels=True, ax=axes[0])
# axes[0].set_title('Node Indices')

# # Plot 2: JX branches
# node_colors = np.zeros(len(jx_subgraph.nodes()))
# for i, branch in enumerate(diff_jx_branches):
#     node_colors[np.isin(list(jx_subgraph.nodes()), branch)] = i + 1
# nx.draw(jx_subgraph, pos=pos, node_color=node_colors, cmap='tab10', with_labels=True, ax=axes[1])
# axes[1].set_title('JX Branches')

# # Plot 3: Morph branches
# node_colors = np.zeros(len(jx_subgraph.nodes()))
# for i, branch in enumerate(diff_morph_branches):
#     node_colors[np.isin(list(jx_subgraph.nodes()), branch)] = i + 1
# nx.draw(jx_subgraph, pos=pos, node_color=node_colors, cmap='tab10', with_labels=True, ax=axes[2])
# axes[2].set_title('Morph Branches')

# plt.tight_layout()
# plt.show()